In [30]:

import mxnet as mx
import numpy as np
import logging

logging.basicConfig(level=logging.INFO)

sample_count = 1000
train_count = 800
valid_count = sample_count - train_count

feature_count = 100
category_count = 10
batch=10

X = mx.nd.uniform(low=0, high=1, shape=(sample_count,feature_count))
Y = mx.nd.empty((sample_count,))
for i in range(0,sample_count-1):
  Y[i] = np.random.randint(0,category_count)

X_train = mx.nd.crop(X, begin=(0,0), end=(train_count,feature_count))
Y_train = Y[0:train_count]

X_valid = mx.nd.crop(X, begin=(train_count,0), end=(sample_count,feature_count))
Y_valid = Y[train_count:sample_count]

#print(X.shape, Y.shape, X_train.shape, Y_train.shape, X_valid.shape, Y_valid.shape)

# Build network
data = mx.sym.Variable('data')
fc1 = mx.sym.FullyConnected(data, name='fc1', num_hidden=64)
relu1 = mx.sym.Activation(fc1, name='relu1', act_type="relu")
fc2 = mx.sym.FullyConnected(relu1, name='fc2', num_hidden=category_count)
out = mx.sym.SoftmaxOutput(fc2, name='softmax')
mod = mx.mod.Module(out)

# Build iterator
train_iter = mx.io.NDArrayIter(data=X_train,label=Y_train,batch_size=batch)
#for batch in train_iter:
#  print batch.data
#  print batch.label

# Train model
mod.bind(data_shapes=train_iter.provide_data, label_shapes=train_iter.provide_label)
mod.init_params(initializer=mx.init.Xavier(magnitude=2.))
mod.init_optimizer(optimizer='sgd', optimizer_params=(('learning_rate', 0.1), ))
mod.fit(train_iter, num_epoch=60)

#pred_iter = mx.io.NDArrayIter(data=X_train,label=Y_train, batch_size=batch)
#pred_count = train_count
pred_iter = mx.io.NDArrayIter(data=X_valid,label=Y_valid, batch_size=batch)
pred_count = valid_count

correct_preds = total_correct_preds = 0
print('batch [labels] [predicted labels]  correct predictions')
for preds, i_batch, batch in mod.iter_predict(pred_iter):
    label = batch.label[0].asnumpy().astype(int)
    pred_label = preds[0].asnumpy().argmax(axis=1)
    correct_preds = np.sum(pred_label==label)
    print(i_batch, label, pred_label, correct_preds)
    total_correct_preds = total_correct_preds + correct_preds

print('Validation accuracy: %2.2f' % (1.0*total_correct_preds/pred_count))


/home/anm/anaconda3/lib/python3.6/site-packages/mxnet/module/base_module.py:488: UserWarning: Parameters already initialized and force_init=False. init_params call ignored.
  allow_missing=allow_missing, force_init=force_init)


batch [labels] [predicted labels]  correct predictions
0 [9 5 3 1 4 3 0 4 9 2] [8 3 2 8 8 3 5 7 0 7] 1
1 [9 6 6 1 0 1 7 8 9 2] [8 7 1 7 2 6 4 8 6 7] 1
2 [5 5 3 2 2 8 4 9 7 3] [2 5 3 0 8 1 3 0 8 0] 2
3 [7 0 9 8 6 2 3 9 3 0] [3 8 3 5 5 9 4 3 0 7] 0
4 [6 3 5 0 7 8 5 1 8 9] [4 3 3 4 5 1 5 7 8 7] 3
5 [9 8 7 6 0 2 5 0 0 5] [4 1 4 1 3 7 8 9 4 4] 0
6 [8 9 0 7 6 5 3 1 2 6] [7 5 9 7 2 1 8 2 8 7] 1
7 [7 0 7 0 2 0 2 6 1 9] [7 4 3 3 9 8 8 6 8 6] 2
8 [3 7 4 9 1 0 4 8 0 2] [1 8 8 6 8 4 8 4 5 6] 0
9 [0 5 3 3 1 0 3 2 2 1] [7 0 9 7 5 7 8 7 5 8] 0
10 [2 6 3 2 4 7 2 6 4 4] [5 4 6 6 8 2 9 5 6 6] 0
11 [3 1 8 9 3 4 9 7 0 4] [1 1 8 1 7 8 8 3 7 1] 2
12 [4 3 0 2 3 8 4 1 8 8] [7 8 3 1 1 9 3 5 6 7] 0
13 [4 2 4 8 1 6 3 7 7 6] [8 5 1 8 3 7 3 8 7 4] 3
14 [6 1 6 5 5 9 5 5 5 7] [6 8 8 0 2 7 6 8 3 2] 1
15 [2 1 1 8 3 1 1 0 3 7] [4 1 3 3 5 1 3 7 6 1] 2
16 [5 9 2 7 4 7 8 5 8 9] [3 6 3 1 5 8 1 6 0 5] 0
17 [8 7 2 8 0 5 6 9 4 3] [6 1 0 3 5 7 1 4 8 8] 0
18 [7 8 4 0 9 7 0 8 7 1] [8 8 2 5 5 1 6 8 7 1] 4
19 [4 8 9 7 2 8 7 3 8 0]